# MINI Project 2 - Hybrid Encoder Model

## Group Members:

```
IIT2018079 - Kisalaya Kishore
IIT2018176 - Milan Bhuva
IIT2018178 - Manav Agrawal
IIT2018179 - Mohammed Aadil
IIT2018202 - Ankit Rauniyar
```

### INDEX : 

**2)  [Mount G-Drive](#scrollTo=Jm42Bj4po3Y-)**<br/>
**2)  [VGG-16](#scrollTo=-R2hznFbcZwk)**<br/>
**3)  [ResNet-50](#scrollTo=5WSDqIUicZwl)**<br/>
**4)  [EffecientNet-V3](#scrollTo=NsSwcr7fcZwm)**<br/>
**5)  [Fusion](#scrollTo=y-G7MA4OcZwn)**<br/>

------

### Header files

In [ ]:
# our utils file
import utils
from models import VGG, ResNet, EffNetB3

# basic ML
import os
import numpy as np
import matplotlib.pyplot as plt

# connect to drive
from google.colab import drive

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import svm

### Mounting G-Drive

In [ ]:
drive.mount('/content/gdrive')
pkldest = '/content/gdrive/MyDrive/pkl_files/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


------

# Model 2

<img src="https://user-images.githubusercontent.com/66634743/114007474-70237e00-9872-11eb-9505-5ddb848e9d01.png" height=400 width=800>

### Loading the Labels

In [ ]:
y = utils.load_from_pkl("scenes.pkl")

------

## VGG-16

In [ ]:
vgg_features = utils.load_from_pkl("features/vgg16_features.pkl")

encoder_vgg = encoder("VGG")
encoder_vgg.encode(vgg_features)

vggX = encoder_vgg.encodeTrainer()

### Classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vggX, y, test_size=0.2, random_state=42)

classifier = svm.SVC(kernel='rbf')

classifier.fit(X_train, y_train)

### Temporary Results

In [ ]:
pred = classifier.predict(X_test)

acc = np.sum(1*(pred==y_test))/y_test.shape[0]
print("vgg_16 + encoder+SVM  = ", acc*100)

## ResNet50

In [ ]:
resnetfeatures=utils.load_from_pkl("features/resnet_features.pkl")

encoder_resnet=encoder("Resnet")
encoder_resnet.encode(resnetfeatures)
resnetX=encoder_resnet.encodeTrainer()

### Classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(resnetX,y, test_size=0.2, random_state=42)

classifier = svm.SVC(kernel='rbf')

classifier.fit(X_train, y_train)

### Temporary Results

In [ ]:
pred = classifier.predict(X_test)

acc = np.sum(1*(pred==y_test))/y_test.shape[0]
print("Resnet50 + encoder + SVM  = ", acc*100)

## EfficientNetB3

In [ ]:
effnetfeatures=utils.load_from_pkl("features/effnetB3_features.pkl")

encoder_effnet=encoder("effnet")
encoder_effnet.encode(effnetfeatures)
effnetX=encoder_effnet.encodeTrainer()

### Classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(effnetX,y, test_size=0.2, random_state=42)

classifier = svm.SVC(kernel='rbf')

classifier.fit(X_train, y_train)

### Temporary Results

In [ ]:
pred = classifier.predict(X_test)

acc = np.sum(pred==y_test)/y_test.shape[0]
print("EffnetB3 + encoder + SVM  = ", acc*100)

## Fusing the predictions of the 3 models

In [ ]:
Xstacked=np.hstack((effnetX, vggX, resnetX))

X_train, X_test, y_train, y_test = train_test_split(Xstacked, y, test_size=0.2, random_state=42)

### Final Classifier

In [ ]:
classifier = svm.SVC(kernel='rbf')
classifier.fit(X_train,y_train)
ypred = classifier.predict(X_test)

### Final Model 2 Results

In [ ]:
acc = (np.sum(ypred==y_test))/y_test.shape[0]

print("Acc of Model 2 : ", acc)

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test,ypred)
sns.heatmap(cm)